In [0]:
%sh
# clear the delta checkpoint
rm -rf /dbfs/tmp/StreamingDataQuality/checkpoint

# download some generated stock tick data; this is a public Mockaroo endpoint- as such, we can't guarantee availability!
curl "https://api.mockaroo.com/api/2aedaa80?count=1000&key=8eb06b50" > /tmp/stockTicks.json


% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 30301 0 30301 0 0 26099 0 --:--:-- 0:00:01 --:--:-- 26076
100 100k 0 100k 0 0 50585 0 --:--:-- 0:00:02 --:--:-- 50560
100 149k 0 149k 0 0 63362 0 --:--:-- 0:00:02 --:--:-- 63362

In [0]:
dbutils.fs.mv("file:/tmp/stockTicks.json", "dbfs:/tmp/StreamingDataQuality/stockTicks.json")

Out[2]: True

In [0]:
%pip install git+https://github.com/tdoehmen/duckdq

Python interpreter will be restarted.
Collecting git+https://github.com/tdoehmen/duckdq
 Cloning https://github.com/tdoehmen/duckdq to /tmp/pip-req-build-jjfq78nz
 Running command git clone -q https://github.com/tdoehmen/duckdq /tmp/pip-req-build-jjfq78nz
Collecting duckdb>=0.2.3
 Downloading duckdb-0.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.1 MB)
Collecting SQLAlchemy
 Downloading SQLAlchemy-2.0.29-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from duckdq==0.0.post0.dev9+g5032943) (1.2.4)
Collecting whylabs-datasketches
 Downloading whylabs_datasketches-2.2.0b1-cp38-cp38-manylinux2010_x86_64.whl (405 kB)
Requirement already satisfied: dill in /databricks/python3/lib/python3.8/site-packages (from duckdq==0.0.post0.dev9+g5032943) (0.3.2)
Collecting tryingsnake
 Downloading tryingsnake-0.5.1.tar.gz (13 kB)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->duckdq==0.0.post0.dev9+g5032943) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas->duckdq==0.0.post0.dev9+g5032943) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->duckdq==0.0.post0.dev9+g5032943) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas->duckdq==0.0.post0.dev9+g5032943) (1.15.0)
Collecting typing-extensions>=4.6.0
 Downloading typing_extensions-4.11.0-py3-none-any.whl (34 kB)
Collecting greenlet!=0.4.17
 Downloading greenlet-3.0.3-cp38-cp38-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (622 kB)
Building wheels for collected packages: duckdq, tryingsnake
 Building wheel for duckdq (setup.py): started
 Building wheel for duckdq (setup.py): finished with status 'done'
 Created wheel for duckdq: filename=duckdq-0.0.post0.dev9+g5032943-py2.py3-none-any.whl size=43626 sha256=073ddc4b99c8c3900882b22382583d6b97aa4174ac9d8a65e6408e924caa483f
 Stored in directory: /tmp/pip-ephem-wheel-cache-p5mrmawj/wheels/88/a5/74/5b722e499bbd67855336523f1b88d3078e69eeb50f563fbfce
 Building wheel for tryingsnake (setup.py): started
 Building wheel for tryingsnake (setup.py): finished with status 'done'
 Created wheel for tryingsnake: filename=tryingsnake-0.5.1-py3-none-any.whl size=11427 sha256=a8862ce1b7272d8adc3a9cb5d5864285025ec27ba4ee87df215c762099a2ef6a
 Stored in directory: /root/.cache/pip/wheels/0a/e8/c5/7eda97b4220dd6b8cc007d52a6f41aae0e64b1d9a5572dfa28
Successfully built duckdq tryingsnake
Installing collected packages: typing-extensions, greenlet, whylabs-datasketches, tryingsnake, SQLAlchemy, duckdb, duckdq
 Attempting uninstall: typing-extensions
 Found existing installation: typing-extensions 3.7.4.3
 Not uninstalling typing-extensions at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-9132a3ff-5d8b-46ff-8acb-198c4bf391a5
 Can't uninstall 'typing-extensions'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.11.0 which is incompatible.
Successfully installed SQLAlchemy-2.0.29 duckdb-0.10.1 duckdq-0.0.post0.dev9+g5032943 greenlet-3.0.3 tryingsnake-0.5.1 typing-extensions-4.11.0 whylabs-datasketches-2.2.0b1
WARNING: You are using pip version 21.0.1; however, version 24.0 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9132a3ff-5d8b-46ff-8acb-198c4bf391a5/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from duckdq.checks import Check, CheckLevel

In [0]:
%scala
import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.concat

val data_path = "/tmp/StreamingDataQuality/source/"
val checkpoint_path = "/tmp/StreamingDataQuality/checkpoint/"
val base_df = spark.read.parquet(data_path)
val empty_df = base_df.where("0 = 1")
val l1: Long = 0

spark.sql("DROP TABLE IF EXISTS trades_delta")
spark.sql("DROP TABLE IF EXISTS bad_records")
spark.sql("DROP TABLE IF EXISTS duck_metrics")

base_df.createOrReplaceTempView("trades_historical")
empty_df.write.format("delta").saveAsTable("trades_delta")
empty_df.withColumn("batchID",lit(l1)).write.format("delta").saveAsTable("bad_records")
dbutils.fs.mkdirs(checkpoint_path)

import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.concat
data_path: String = /tmp/StreamingDataQuality/source/
checkpoint_path: String = /tmp/StreamingDataQuality/checkpoint/
base_df: org.apache.spark.sql.DataFrame = [buysell: string, date: string ... 6 more fields]
empty_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [buysell: string, date: string ... 6 more fields]
l1: Long = 0
res0: Boolean = true

In [0]:
import time
import pandas as pd
from duckdq.checks import Check, CheckLevel
from duckdq.verification_suite import VerificationSuite
from duckdq.verification_suite import VerificationResult, VerificationSuite
from duckdq.engines.state_engine import StateEngine
from duckdq.utils.analysis_runner import AnalyzerContext
from duckdq.utils.exceptions import StateMergingException
from duckdq.verification_suite import VerificationResult, VerificationSuite
import duckdb
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import lit, current_timestamp


df = spark.read.json("/tmp/StreamingDataQuality/stockTicks.json")
df = df.toPandas()

verificationResult = (
    VerificationSuite()
    .on_data(df)
    .add_check(
    Check(CheckLevel.WARNING, "Basic Check 2")
    .is_complete("ipaddr")
    .is_complete("quantity")
    .is_complete("price")
    .has_max("quantity", lambda mx: mx <= 10000)
    .is_non_negative("quantity")
        )
        .run()
)
  
data = []  # This will hold our extracted data

for check, check_result in verificationResult.check_results.items():
    check_description = check.description
    for constraint_result in check_result.constraint_results:
        constraint_s = constraint_result.constraint.__str__()
        constraint_result_s = constraint_result.status.name
        metric_value = constraint_result.metric.value
        # Assuming `metric_value.get()` retrieves the actual value and is the standard success path
        metric_s = metric_value.get() if metric_value.isSuccess else "Error retrieving metric"
        # Append extracted information as a dictionary to our data list
        data.append({
            "Check": check_description,
            "Constraint": constraint_s,
            "Status": constraint_result_s,
            "Metric": metric_s
        })

schema = StructType([
    StructField("Check", StringType(), True),
    StructField("Constraint", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Metric", StringType(), True)  # Use FloatType() or DoubleType() if applicable
])


ver_df = spark.createDataFrame(data, schema=schema)

# Assuming you have a way to convert your metrics result to a DataFrame
metric_results = ver_df # Your logic to convert analysis results to DataFrame
metric_results.withColumn("ts", current_timestamp()) \
    .write.format("delta").mode("append").saveAsTable("duck_metrics")




In [0]:
from pyspark.sql.session import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import current_timestamp
import pandas as pd
from pyspark.sql.functions import col

df = spark.table("duck_metrics")
df = df.withColumn("Metric", col("Metric").cast(DoubleType()))
df_filtered = df.filter(col("Constraint").startswith("CompletenessConstraint"))
display(df_filtered)


Check,Constraint,Status,Metric,ts
Basic Check 2,CompletenessConstraint(Completeness(quantity)),FAILURE,0.952,2024-04-09T20:04:13.798+0000
Basic Check 2,CompletenessConstraint(Completeness(ipaddr)),FAILURE,0.965,2024-04-09T20:04:13.798+0000
Basic Check 2,CompletenessConstraint(Completeness(price)),FAILURE,0.986,2024-04-09T20:04:13.798+0000


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.